Creating a Graph/Model

In [2]:
import onnx_graphsurgeon as gs
import onnxruntime as rt
import numpy as np
import onnx

X = gs.Variable(name="X", dtype=np.float32, shape=(3,3))
Y = gs.Variable(name="Y", dtype=np.float32, shape=(2,3))
node = gs.Node(op="Relu", inputs=[X], outputs=[Y])

graph = gs.Graph(nodes=[node], inputs=[X], outputs=[Y])
onnx.save(gs.export_onnx(graph), "./models/test_Relu.onnx")

In [3]:
onnx.checker.check_model("./models/test_Relu.onnx")

In [4]:
graph = gs.import_onnx(onnx.load("./models/test_Relu.onnx"))

input_name = graph.inputs[0].shape
output_name = graph.outputs[0].shape

print(input_name, output_name)


[3, 3] [2, 3]


In [5]:
sess = rt.InferenceSession("./models/test_Relu.onnx")

input_data = np.random.uniform(-1,1,size=(3,3)).astype(np.float32)
print(input_data)
result = sess.run(None, {"X": input_data})
print(np.array(result))

[[-0.55743694  0.39447096  0.18217166]
 [-0.9242534   0.46393532 -0.27336538]
 [ 0.43655092 -0.39844665  0.3889202 ]]
[[[0.         0.39447096 0.18217166]
  [0.         0.46393532 0.        ]
  [0.43655092 0.         0.3889202 ]]]


In [6]:
print(graph.outputs)

[Variable (Y): (shape=[2, 3], dtype=float32)]


In [7]:

Y_intermediate = gs.Variable(name="Y_intermediate", dtype=np.float32, shape=(3, 3))
Y_final = gs.Variable(name="Y", dtype=np.float32, shape=(2, 3))

shape_tensor = gs.Constant(name="shape_tensor", values=np.array([1, 9], dtype=np.int64))

relu_node = gs.Node(op="Relu", inputs=[X], outputs=[Y_intermediate])
reshape_node = gs.Node(op="Reshape", inputs=[Y_intermediate,shape_tensor], outputs=[Y_final])


graph = gs.Graph(nodes=[relu_node,reshape_node], inputs=[X], outputs=[Y_final])
onnx.save(gs.export_onnx(graph), "./models/test_Relu_shape.onnx")

onnx.checker.check_model("./models/test_Relu_shape.onnx")

sess = rt.InferenceSession("./models/test_Relu_shape.onnx")

input_data = np.random.uniform(-1, 1, size=(3, 3)).astype(np.float32)
print(input_data)
result = sess.run(None, {"X": input_data})
print(np.array(result))

[[ 0.20610574 -0.5023196  -0.80173314]
 [-0.9715784   0.9085708  -0.57180136]
 [-0.07007944 -0.60682905 -0.7079372 ]]
[[[0.20610574 0.         0.         0.         0.9085708  0.
   0.         0.         0.        ]]]


### Isolating a subgraph

In [8]:
from onnx import shape_inference
model = onnx.load("./models/resnet50.onnx")

graph = gs.import_onnx(model)

print([subgraph for subgraph in graph.subgraphs()])

[]


In [9]:

import onnx_graphsurgeon as gs
import numpy as np
import onnx

# Computes Y = S + (W * X + B)

shape = (1, 3, 64, 64)

S = gs.Variable(name="S", dtype=np.float32, shape=shape)
X = gs.Variable(name="X", dtype=np.float32, shape=shape)

W = gs.Constant("W", values=np.random.rand(*shape))
B= gs.Constant("B", values=np.random.rand(*shape))

w_x = gs.Variable(name="w_x")
add_bias = gs.Variable(name="add_bias")


Y = gs.Variable(name="Y", dtype=np.float32, shape=shape)

nodes = [
  
    gs.Node(op="Mul", inputs=[W, X], outputs=[w_x]),
   
    gs.Node(op="Add", inputs=[w_x, B], outputs=[add_bias]),

    gs.Node(op="Add", inputs=[add_bias, S], outputs=[Y]),
]

graph = gs.Graph(nodes=nodes, inputs=[S, X], outputs=[Y])
onnx.save(gs.export_onnx(graph), "model.onnx")

### Basics

###### Import and Export ONNX

In [10]:
#Import and Export ONNX

# Takes a graph as input and returns an ONNX model as output

mymodel=gs.export_onnx(graph)  # Export the graph to an ONNX model
onnx.save(mymodel, "./models/test_relu.onnx")  # Save the ONNX model to a file

# Takes an ONNX model as input and returns a graph as output
mygraph=gs.import_onnx(mymodel)  # Import the ONNX model as a graph
mygraph.nodes

[ (Mul)
 	Inputs: [
 		Constant (W): (shape=[1, 3, 64, 64], dtype=float64)
 		Variable (X): (shape=[1, 3, 64, 64], dtype=float32)
 	]
 	Outputs: [
 		Variable (w_x): (shape=None, dtype=None)
 	],
  (Add)
 	Inputs: [
 		Variable (w_x): (shape=None, dtype=None)
 		Constant (B): (shape=[1, 3, 64, 64], dtype=float64)
 	]
 	Outputs: [
 		Variable (add_bias): (shape=None, dtype=None)
 	],
  (Add)
 	Inputs: [
 		Variable (add_bias): (shape=None, dtype=None)
 		Variable (S): (shape=[1, 3, 64, 64], dtype=float32)
 	]
 	Outputs: [
 		Variable (Y): (shape=[1, 3, 64, 64], dtype=float32)
 	]]

###### Graph

<span style="font-size: 14px;">
Represents a graph containing nodes and tensors.

Parameters:
- nodes (Sequence[Node]) – A list of the nodes in this graph.
- inputs (Sequence[Tensor]) – A list of graph input Tensors.
- outputs (Sequence[Tensor]) – A list of graph output Tensors.
- name (str) – The name of the graph. Defaults to “onnx_graphsurgeon_graph”.
</span>
